In [2]:
import os
import re
import csv
import glob
import time
from collections import OrderedDict
import pandas as pd

## read female figure skater (女性フィギュアスケート選手）articles data downloaded from PetScan (https://petscan.wmflabs.org/?psid=22393600)
### User manual (https://meta.wikimedia.org/wiki/PetScan/)


In [4]:
df = pd.read_csv("femaleFigureSkater.csv")
df.head(20)

,number,title,pageid,namespace,length,touched
0,1,河合彩,42650,NaN,7153,20220615015051
1,2,大石恵,71699,NaN,5589,20220416014108
2,3,伊藤みどり,117453,NaN,62989,20220608195210
3,4,中野友加里,176250,NaN,49548,20220524001252
4,5,八木沼純子,190438,NaN,15242,20220622091355
5,6,ジャネット・リン,223139,NaN,11255,20220207144942
6,7,サーシャ・コーエン,223778,NaN,35834,20211122024555
7,8,カロリーナ・コストナー,229105,NaN,59486,20211031034731
8,9,石田治子,242214,NaN,5697,20211112032332
9,10,福原美和,242229,NaN,4805,20220603051245


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2525 entries, 0 to 2524
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   number     2525 non-null   int64  
 1   title      2525 non-null   object 
 2   pageid     2525 non-null   int64  
 3   namespace  0 non-null      float64
 4   length     2525 non-null   int64  
 5   touched    2525 non-null   int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 118.5+ KB


In [6]:
# key: folder's name, value: create dictionartris of max IDs
# exrtract max ID values from the last article in the last file in each directory
def mapping_dir(dir_path):
    dir_list = os.listdir(dir_path)
    for _dir in dir_list:
        last_f_path = sorted(glob.glob(os.path.join(dir_path, _dir) + '\\*'))[-1]
        max_id, _, _ = format_text(sorted(get_articlelist(last_f_path))[-1])
        dir_info[_dir] = max_id
    return dir_info

# pade_id from csv file and dir_info created by mapping_dir
# return folder name which include page_id I'm looking for
def get_search_dir(page_id, dir_info):
    min_id = 0
    for dir_name, max_id in dir_info.items():
        search_dir = dir_name
        if min_id <= page_id <= max_id:
            return search_dir
        else:
            min_id = max_id
    return next(iter(dir_info))     # if not found, retuen the first value(AA)
 
# If nothing is found with search_dir, search again from the beginning
# this can be replaced by "get_files"
def get_all_files(base_path, search_dir):
    dir_iter = iter(dir_info.keys())   # in case search_dir fails
    dir_list = list(dir_info.keys())   # list of key
    cnt = 1
    while True:
        search_dir_path = os.path.join(base_path, search_dir)
        print(search_dir_path)
        for _, _, files in os.walk(search_dir_path):
            for file in files:
                yield os.path.join(search_dir_path, file)
        search_dir = next(dir_iter)
        cnt += 1
        if search_dir == dir_list[-1] and cnt == len(dir_list):
            # when nothing found
            return

In [7]:
# 
def get_files(base_path, search_dir):
    while True:
        search_dir_path = os.path.join(base_path, search_dir)
        print(search_dir_path)
        for _, _, files in os.walk(search_dir_path):
            for file in files:
                yield os.path.join(search_dir_path, file)
        else:
            return
        
# return each article text
def format_text(data):
    head = head_reg.search(data)  # (<doc id=...>)
    id_tag = id_reg.search(head.group())
    doc_id = re.search(r'\d+', id_tag.group())  # extract id from header
    title_tag = title_reg.search(head.group())
    doc_title = re.search(r'".+"', title_tag.group())  
    doc_id = int(doc_id.group())
    title = doc_title.group().replace('"', '')      # delete ""
    # deete each header(<doc id=...>)and tag
    # extract only texts (delete title as some articles have duplicated tittles)
    text = data.replace(head.group(), '').rstrip('</doc>')[len(title):]
    return doc_id, title, text

# return lists of articles
def get_articlelist(f_path):
    f = open(f_path, 'r', encoding='utf-8')
    file_data = f.read()
    file_data = file_data.replace('\n', '')
    return doc_reg.findall(file_data)

In [8]:
def output_file(f_name, text):
    # delete special characters which can't be used for file names
    f_name = re.sub(r'¥|/|:|\*|\?|"|<|>|\||', '', f_name)
    path = OUTPUT_PATH + f_name
    f = open(path, 'w', encoding='utf-8')
    f.write(text)
    f.close()
    
def output_file(f_name, text):
    # delete special characters which can't be used for file names
    f_name = re.sub(r'¥|/|:|\*|\?|"|<|>|\||', '', f_name)
    path = OUTPUT_PATH + f_name
    f = open(path, 'w', encoding='utf-8')
    f.write(text)
    f.close()
    
if __name__ == '__main__':
    # 
    id_reg = re.compile(r'id="\d+"')
    title_reg = re.compile(r'title=".+"')
    doc_reg = re.compile(r'<doc id=.+?</doc>')
    head_reg = re.compile(r'<doc id.+?">')

### NEED TO CHANGE for differendt data

In [9]:
BASE_PATH = 'F:/Wikipedia/wikiextractor-master/wikiextractor'
CSV_FILE = BASE_PATH + '/femaleFigureSkater.csv' # downloaded data from PetScan
WIKI_DIR = BASE_PATH + '/text'
OUTPUT_PATH = 'F:/Wikipedia/wikiextractor-master/wikiextractor/femaleFigureSkater/'

In [10]:
# make dictionary of the max ID value in each folder
dir_info = OrderedDict()
dir_info = mapping_dir(WIKI_DIR)
counter = 1     # loop counter

In [11]:
# read csv
f = open(CSV_FILE, 'r', encoding='utf-8')
reader = csv.reader(f)
next(csv.reader(f))

['number', 'title', 'pageid', 'namespace', 'length', 'touched']

In [12]:
not_found = {}      # dict for not-found data

In [13]:
# measure time
start_time = time.time()

In [14]:
for line in reader:
        flag = False
        page_name = line[1]
        page_id = int(line[2])
        namespace = line[3]
        if namespace == "Category" or namespace == "Template":
            continue
        search_dir = get_search_dir(page_id, dir_info)
        for f_path in get_files(WIKI_DIR, search_dir):
            for d in get_articlelist(f_path):
                doc_id, title, text = format_text(d)
                if page_id == doc_id:
                    print(counter, title)
                    output_file(title, text)
                    counter += 1
                    flag = True
                    break
            if flag:
                break
        else:
            not_found[page_id] = page_name
            print("ページID: {}は見つかりませんでした.".format(page_id))
 
        end_time = time.time()
        processing_time = end_time - start_time
        print('processing_time(sec): ', processing_time)
 
        # print "not found" page and its ID
        for i, (_id, _title) in enumerate(not_found.items()):
            print(i, _id, _title)

F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
1 河合彩
processing_time(sec):  3.263124704360962
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
2 大石恵
processing_time(sec):  4.549963474273682
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AE
3 伊藤みどり
processing_time(sec):  4.677922248840332
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AF
4 中野友加里
processing_time(sec):  5.068485975265503
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AF
5 八木沼純子
processing_time(sec):  5.705476522445679
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AG
6 ジャネット・リン
processing_time(sec):  5.805378675460815
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AG
7 サーシャ・コーエン
processing_time(sec):  5.8876683712005615
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AG
8 カロリーナ・コストナー
processing_time(sec):  6.11124587059021
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AG
9 石田治子
processing_time(sec):  6.513668537139893
F:/Wikipedia/wikiextractor-master/wikiextractor

77 タニス・ベルビン
processing_time(sec):  60.415422439575195
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AM
78 ベンジャミン・アゴスト
processing_time(sec):  61.51105308532715
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AM
79 坂頂みなみ
processing_time(sec):  62.579591035842896
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AM
80 水津瑠美
processing_time(sec):  63.617268323898315
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AM
81 ジョアニー・ロシェット
processing_time(sec):  64.69455575942993
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AM
82 クリス・リード
processing_time(sec):  66.02072834968567
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AM
83 オクサナ・バイウル
processing_time(sec):  67.81197214126587
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AM
84 ソニア・ヘニー
processing_time(sec):  69.50158762931824
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AM
85 ペギー・フレミング
processing_time(sec):  71.66524529457092
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AM
86 アネット・ペッチ
proce

151 クリストファー・ディーン
processing_time(sec):  138.58647966384888
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
152 ナタリア・リニチュク
processing_time(sec):  139.65925645828247
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
153 ゲンナジー・カルポノソフ
processing_time(sec):  140.70109629631042
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
154 リュドミラ・パホモワ
processing_time(sec):  141.73381280899048
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
155 アレクサンドル・ゴルシコフ
processing_time(sec):  142.7725956439972
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
156 イリーナ・ロドニナ
processing_time(sec):  143.8097653388977
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
157 アレクサンドル・ザイツェフ (フィギュアスケート選手)
processing_time(sec):  144.82325673103333
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
158 アレクセイ・ウラノフ
processing_time(sec):  145.9873776435852
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
159 リュドミラ・ベルソワ
processing_time(sec):  147.13127875328064
F:/Wikipedia/wikiex

224 アンドレイ・ブシュコフ
processing_time(sec):  227.51234483718872
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
225 マンディ・ベッツェル
processing_time(sec):  229.29565834999084
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
226 インゴ・シュトイアー
processing_time(sec):  230.7012813091278
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
227 マリア・ペトロワ
processing_time(sec):  232.06462168693542
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
228 アレクセイ・ティホノフ
processing_time(sec):  233.39590692520142
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
229 アントン・シハルリドゼ
processing_time(sec):  234.96781206130981
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
230 若松詩子
processing_time(sec):  237.01071095466614
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
231 アルベナ・デンコヴァ
processing_time(sec):  238.9978575706482
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
232 伊奈恭子
processing_time(sec):  241.2316927909851
F:/Wikipedia/wikiextractor-master/wikiextractor/text\

298 カトリン・フロイデルスペルガー
processing_time(sec):  393.0181517601013
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AO
299 フェデリカ・ファイエラ
processing_time(sec):  393.2396082878113
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AO
300 マッシモ・スカリ
processing_time(sec):  393.45578813552856
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AO
301 タニヤ・シェフチェンコ
processing_time(sec):  393.6758415699005
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AO
302 ダグマル・ルルツ
processing_time(sec):  393.92756152153015
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AO
303 ハナ・マシュコヴァー
processing_time(sec):  394.1983256340027
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AO
304 レギーネ・ハイツァー
processing_time(sec):  394.4224512577057
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AO
305 バーバラ・ロールズ
processing_time(sec):  394.67335772514343
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AO
306 イングリート・ヴェンドル
processing_time(sec):  394.9389183521271
F:/Wikipedia/wikiextractor-master/wikiext

...

2202 ガリーナ・クハル
processing_time(sec):  2069.34818315506
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2203 モニカ・フェルトマン
processing_time(sec):  2069.4107398986816
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2204 アイリーン・ツィルマー
processing_time(sec):  2069.473219394684
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2205 ペトラ・ルールマン
processing_time(sec):  2069.535710811615
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2206 パトリシア・ドッド
processing_time(sec):  2069.609314918518
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2207 サリー＝アン・ステープルフォード
processing_time(sec):  2069.671769142151
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2208 エリザベート・ネストラー
processing_time(sec):  2069.734302043915
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2209 エリザベート・ミクーラ
processing_time(sec):  2069.790236234665
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2210 フランシス・ウォグホーン
processing_time(sec):  2069.8685796260834
F:/Wikipedia/wikiextractor-mas

2275 パトリシア・ポウリー
processing_time(sec):  2087.582194328308
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2276 ダニエル・リグロ
processing_time(sec):  2088.000014066696
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2277 バルバラ・マルティン
processing_time(sec):  2088.523567676544
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2278 ウルゼル・バルケイ
processing_time(sec):  2088.9609763622284
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2279 カルラ・ティカツケーク
processing_time(sec):  2089.4525434970856
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2280 リリアン・クロザ
processing_time(sec):  2089.9094166755676
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2281 マリアン・セージ
processing_time(sec):  2090.4319636821747
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2282 パトリシア・イーストウッド
processing_time(sec):  2091.1041193008423
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2283 アイリーン・ショウ
processing_time(sec):  2091.558894634247
F:/Wikipedia/wikiextractor-master/wiki

2347 エミー・プツィンガー
processing_time(sec):  2118.667068004608
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2348 グレーテ・ライナー
processing_time(sec):  2119.0922458171844
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2349 ビアンカ・シェンク
processing_time(sec):  2119.550225019455
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2350 フィクトーリア・リントパインター
processing_time(sec):  2119.986424446106
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2351 オードリー・ペッペ
processing_time(sec):  2120.4134163856506
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2352 ルイーズ・ウェイゲル
processing_time(sec):  2120.839138507843
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2353 エステル・ウェイゲル
processing_time(sec):  2121.275772333145
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2354 モリー・フィリップス
processing_time(sec):  2121.6827445030212
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2355 グウェネス・バトラー
processing_time(sec):  2122.12819314003
F:/Wikipedia/wikiextractor-master/

2419 ジェナ・アロースミス
processing_time(sec):  2153.73704123497
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2420 スヴェトラーナ・チェルニショワ
processing_time(sec):  2154.28705906868
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2421 ミシェル・アラール
processing_time(sec):  2154.838321208954
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2422 カレン・クワン
processing_time(sec):  2155.3694019317627
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2423 リュドミラ・バコニナ
processing_time(sec):  2155.9192781448364
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2424 マリーナ・クドリャフツェワ
processing_time(sec):  2156.478750705719
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2425 エレーナ・アレクサンドロワ
processing_time(sec):  2157.0211951732635
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2426 ユリア・ニコラエワ
processing_time(sec):  2157.5844502449036
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
2427 カローラ・ヴァイセンベルク
processing_time(sec):  2158.1525490283966
F:/Wikipedia/wikiextractor-

2493 吉田陽菜
processing_time(sec):  2204.6779057979584
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
2494 スター・アンドリュース
processing_time(sec):  2205.3240609169006
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
2495 松永幸恵
processing_time(sec):  2205.99080824852
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
2496 河辺愛菜
processing_time(sec):  2206.804986476898
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
2497 佐藤操
processing_time(sec):  2207.6652805805206
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
2498 西山真瑚
processing_time(sec):  2208.7993087768555
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
2499 吉田唄菜
processing_time(sec):  2209.805751800537
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
2500 ジュディ・シュヴォマイヤー
processing_time(sec):  2210.848381280899
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
2501 ジェームス・スラドキー
processing_time(sec):  2211.8618664741516
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BG
2502 ダリア・ウサ